# Ex2

In [ ]:
%matplotlib inline

from __future__ import print_function

from numpy import eye, loadtxt, hstack, zeros, ones, dot, transpose, array, linspace, logspace, \
    meshgrid, float64, log, finfo, exp
from matplotlib import cm
from matplotlib.pyplot import plot, ylabel, xlabel, figure, subplots, contour, legend, axis, contour
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize

# Part 1: Plotting

Plotting data with + indicating (y = 1) examples and o indicating (y = 0) examples.

In [ ]:
def plot2_data(X, y, data_labels, axis_labels):
    pos = y[:, 0] == 1
    neg = y[:, 0] == 0

    plot(X[pos, 0], X[pos, 1], 'k+', label=data_labels[0],
       markerfacecolor='b', markersize=7)
    plot(X[neg, 0], X[neg, 1], 'ko', label=data_labels[1],
       markerfacecolor='Y', markersize=7)

    # Put some labels
    if axis_labels and len(axis_labels) > 1:
        xlabel(axis_labels[0])
        ylabel(axis_labels[1])

data = loadtxt('ex2data1.txt', dtype=float64, delimiter=',')
X, y = data[:, :2], data[:, -1].reshape(-1, 1)

plot2_data(X, y,
          ['Admitted', 'Not Admitted'],
          ['Exam 1', 'Exam 2'])
legend(numpoints=1)

# Part 2: Compute Cost and Gradient

In [ ]:
eps = finfo(float64).eps
eps = 1e-5

def sigmoid(z):
    return 1. / (1. + exp(-z))

def cost_function(theta, X, y):
    """cost function"""

    m, n = X.shape
    theta = theta.reshape(n, 1)

    h_theta = sigmoid(X.dot(theta))
    h_theta[h_theta < eps] = eps
    h_theta[(1 - eps < h_theta) & (h_theta < 1 + eps)] = 1 - eps
    J = (- y.T.dot(log(h_theta)) - (1. - y).T.dot(log(1. - h_theta))) / m
    grad = X.T.dot(h_theta - y) / m

    return J[0, 0], grad.flatten()

In [ ]:
# Initialize fitting parameters
initial_theta = zeros((n + 1, 1))

# Setup the data matrix appropriately, and add ones for the intercept term
m, n = X.shape

# Add intercept term to x and X_test
X = hstack((ones((m, 1)), X))

# Compute and display initial cost and gradient
cost, grad = cost_function(initial_theta, X, y)
print('Cost at initial theta (zeros): {}\nGradient at initial theta (zeros): {}'.format(cost, grad))

# Part 3: Optimizing using fminunc

In [ ]:
def plot_decision_boundary(theta, X, y, *args):

    # Plot Data
    plot2_data(X[:, 1:3], y, *args)

    n = X.shape[1]

    if n <= 3:
        # Only need 2 points to define a line, so choose two endpoints
        plot_x = array([min(X[:, 1]) - 2, max(X[:, 1]) + 2])

        # Calculate the decision boundary line
        plot_y = (-1 / theta[2]) * (theta[1] * plot_x + theta[0])

        # Plot, and adjust axes for better viewing
        plot(plot_x, plot_y, label='Decision Boundary')

        # Legend, specific for the exercise
        axis([30, 100, 30, 100])
    else:
        # Here is the grid range
        u = linspace(-1, 1.5, 50) 
        v = linspace(-1, 1.5, 50)
        size = u.size
        z = zeros((size, size))
        # Evaluate z = theta*x over the grid
        for i in range(size):
            for j in range(size):
                z[i, j] = map_feature(u[i], v[j]).dot(theta)

        z = z.T
        contour(u, v, z, [0, 0], linewidth=2, label='Decision Boundary')

In [ ]:
result = minimize(cost_function,
                  initial_theta,
                  args=(X, y),
                  method='BFGS',
                  jac=True,
                  options=dict(maxiter=400))
cost = result.fun
theta = result.x

print('''Cost at theta found by minimize: {}
theta: {}'''.format(cost, theta))

plot_decision_boundary(theta, X, y,
                       ['Admitted', 'Not Admitted'],
                       ['Exam 1', 'Exam 2'])
legend(numpoints=1)